<a href="https://colab.research.google.com/github/esassoc/qanat-community/blob/develop/Qanat.CommunityAPI/Examples/Hackathon_2025_Use_Case_1_Choropleth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Download API documentation
import requests

try:
    url = "https://raw.githubusercontent.com/esassoc/qanat-community/develop/Qanat.CommunityAPI/Examples/groundwater-accounting-platform-api.json"
    response = requests.get(url)
    response.raise_for_status() # Raise an exception for bad status codes

    with open("groundwater-accounting-platform-api.json", "w") as f:
        f.write(response.text)

    # Load API documentation
    import json

    with open('groundwater-accounting-platform-api.json', 'r') as f:
        api_documentation = json.load(f)

    # API key setup
    from google.colab import userdata

    api_key = None  # Initialize api_key to None

    try:
      api_key = userdata.get('API_KEY')
      if not api_key:
          # If the secret exists but is empty, print message and continue to check api_key
          print("API key found in Colab Secrets but is empty. Please provide your API key.")
          api_key = None # Ensure api_key is None if empty

    except userdata.SecretNotFoundError:
      # If the secret does not exist, print instructions and continue to check api_key
      print("-----------------------------------------------------------------------")
      print("API key not found in Colab Secrets.")
      print("Please add your API key to Colab Secrets:")
      print("1. Click on the '🔑' icon in the left sidebar.")
      print("2. Click on 'New secret'.")
      print("3. For 'Name', enter 'API_KEY'.")
      print("4. For 'Value', paste your API key.")
      print("5. Click 'Save secret'.")
      print("Then, run this cell again.")
      print("-----------------------------------------------------------------------")
      api_key = None # Ensure api_key is None if not found

    # Only proceed with API call if API key is available
    if api_key:
        print("API key successfully loaded from Colab Secrets.")

        # Initial API call to list geographies
        base_url = api_documentation['servers'][0]['url'].rstrip('/')
        geographies_path = '/geographies'

        geographies_url = f"{base_url}{geographies_path}"

        headers = {
            "x-api-key": api_key
        }

        print(f"Attempting to call: {geographies_url}")
        response = requests.get(geographies_url, headers=headers)

        if response.status_code == 200:
          geographies_data = response.json()
          print("Available Geographies:")
          print(json.dumps(geographies_data, indent=2))

          # Extract a geography_id from the response to use for the next API call
          if geographies_data and isinstance(geographies_data, list) and len(geographies_data) > 0:
              geography_id = geographies_data[0].get('GeographyID') # Corrected key to 'GeographyID'
              if geography_id:
                  print(f"Using geography_id: {geography_id} for subsequent calls.")
              else:
                  print("Could not find 'GeographyID' in the first geography entry.")
                  geography_id = None # Set geography_id to None if not found
          else:
              print("No geographies found in the response or unexpected response format.")
              geography_id = None # Set geography_id to None if no geographies found

          # Only proceed with the next API call if a geography_id was found
          if geography_id:
            # Construct the URL for the water measurement types endpoint
            water_measurement_types_path = f'/geographies/{geography_id}/water-measurement-types'
            water_measurement_types_url = f"{base_url}{water_measurement_types_path}"

            print(f"Attempting to call: {water_measurement_types_url}")

            # Make the API call
            response = requests.get(water_measurement_types_url, headers=headers)

            # Process the response
            if response.status_code == 200:
                water_measurement_types_data = response.json()
                print("Available Water Measurement Types:")
                print(json.dumps(water_measurement_types_data, indent=2))
            else:
                print(f"Error: API call failed with status code {response.status_code}")
                print(response.text)
                if response.status_code == 401:
                    print("Authentication Error: Please check your API key in Colab secrets ('API_KEY').")
          else:
              print("Skipping water measurement types API call as no valid geography_id was found.")

        else:
          print(f"Error: API call failed with status code {response.status_code}")
          print(response.text)
          if response.status_code == 401:
              print("Authentication Error: Please check your API key in Colab secrets ('API_KEY').")

except requests.exceptions.RequestException as e:
  print(f"Error: An error occurred during the API request: {e}")

API key successfully loaded from Colab Secrets.
Attempting to call: http://api.groundwateraccounting.org/geographies
Available Geographies:
[
  {
    "GeographyID": 5,
    "GeographyName": "Demo",
    "GeographyDisplayName": "Demo Geography"
  }
]
Using geography_id: 5 for subsequent calls.
Attempting to call: http://api.groundwateraccounting.org/geographies/5/water-measurement-types
Available Water Measurement Types:
[
  {
    "WaterMeasurementTypeID": 5,
    "WaterMeasurementTypeName": "OpenET Evapotranspiration",
    "WaterMeasurementCategoryType": "ET",
    "IsActive": true,
    "GeographyID": 5
  },
  {
    "WaterMeasurementTypeID": 13,
    "WaterMeasurementTypeName": "OpenET Precipitation",
    "WaterMeasurementCategoryType": "Precip",
    "IsActive": true,
    "GeographyID": 5
  },
  {
    "WaterMeasurementTypeID": 19,
    "WaterMeasurementTypeName": "Consumed Groundwater",
    "WaterMeasurementCategoryType": "Calculated",
    "IsActive": true,
    "GeographyID": 5
  },
  {
    